In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import os
import numpy as np
import sys
import scipy
import random
import math
import skimage.io
import datetime
import open3d as o3d

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

Capsules_DIR = os.path.join(ROOT_DIR, "datasets/capsules")


import capsules

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
CAPSULES_MODEL_PATH = "../../mask_rcnn_capsule_0100.h5"

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

from poke_grasp.msg import stone_pose
#from dig-grasping.msg import stone_pose
import rospy
import geometry_msgs.msg
import time
import actionlib
from std_msgs.msg import String

In [ ]:
rospy.init_node('capsule_segmentation')

In [ ]:
def make_directories():
    if not os.path.exists("JPEGImages/"):
        os.makedirs("JPEGImages/")
    if not os.path.exists("depth/"):
        os.makedirs("depth/")

In [ ]:
# Start instance segmentation by Mask RCNN
class InferenceConfig(capsules.CapsulesConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
##config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(CAPSULES_MODEL_PATH, by_name=True)

# Load dataset
# Get the dataset from the releases page
# https://github.com/matterport/Mask_RCNN/releases
dataset = capsules.CapsulesDataset()
dataset.load_capsules(Capsules_DIR, "train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
    
class_names = dataset.class_names

In [ ]:
def pixel_to_camera(pixel, intrin, depth):

    X = (pixel[0]-intrin[0]) * depth / intrin[2]
    Y = (pixel[1]-intrin[1]) * depth / intrin[3]
    return [X, Y]

In [ ]:
is_detect = 0
img_index = 0
def img_index_callback(data):
    global img_index
    global is_detect
    
    print(data.data)
    img_index = data.data
    is_detect = 1

In [ ]:

rospy.Subscriber('/stone_img_index', String, img_index_callback)

In [ ]:
def find_center(mask):
    g = np.mgrid[0:(mask.shape[0]),0:(mask.shape[1])]
    multiple_ = np.stack([mask,mask],0)*g
    total_sum = np.sum(multiple_,axis = (1,2))
    total_number = np.sum(mask)
    average = total_sum/total_number

    return average.astype(int)

In [ ]:
def find_depth(mask,depth_im, depth_value_percentage_taken = 25.0):   
    
    masked_depth_im = depth_im * mask
    #plt.imshow(masked_depth_im)
    #plt.show()
    depth_list = masked_depth_im[masked_depth_im > 0]
    lower_bound = np.percentile(depth_list, 50 - depth_value_percentage_taken/2)
    upper_bound = np.percentile(depth_list, 50 + depth_value_percentage_taken/2)
    
    cut_depth_list1 = depth_list[depth_list >= lower_bound]
    cut_depth_list2 = cut_depth_list1[cut_depth_list1 <= upper_bound]
    
    depth_cut_average = np.average(cut_depth_list2)

    
    depth_max = np.amin(cut_depth_list2)/1000.0
    depth = depth_cut_average/1000.0
    
    
    
    return depth, depth_max

In [ ]:
def find_mask(im,depth_im,r,center_point,distance_axis):
    
    mask_store = 10

    print('center point',center_point)
    center_y, center_x = center_point
    
    mask_size = np.sum(r['masks'],axis = (0,1))
    
    mask_index = np.argsort(np.array(mask_size))[-min(mask_store,r['masks'].shape[2]):]
    
    print('mask_size[mask_index]',mask_size[mask_index])
    
    min_distance2 = 10000000000
    
    most_center_point = []
    
    most_center_mask = []
    
    most_center_depth = []
    
    
    
    for m in mask_index:
        if mask_size[m] < 2000 and mask_size[m] > 1600 : #>1700
        
            mask = r['masks'][:,:,m]
            mask = mask.astype(np.uint8)
            

            ## calculate center

            [contours,hierarchy] = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            cnt = contours[0]
            M = cv2.moments(cnt)
            
            [center_point_y,center_point_x] = find_center(mask)
            

            distance2 = (center_point_x - center_x) **2 *distance_axis[1] + (
                center_point_y -center_y) **2 *distance_axis[0]


            #print(' mask')
            #print('distance2',distance2)
            #plt.imshow(mask)
            #plt.show()



            if distance2 < min_distance2:
                

                depth, depth_max1 = find_depth(mask,depth_im, depth_value_percentage_taken = 25.0)
                depth2, depth_max = find_depth(mask,depth_im, depth_value_percentage_taken = 75.0)


                #print('mask')
                plt.imshow(mask)
                #plt.show()


                kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(50, 50))
                mask_dilated = cv2.dilate(mask, kernel)


                #print('mask_dilated')
                plt.imshow(mask_dilated)
                #plt.show()

                depth_env, depth_max_env = find_depth(mask_dilated,depth_im, depth_value_percentage_taken = 25.0)

                print('depth',depth)
                print(' depth_max_env', depth_max_env)
                #if depth_max_env > depth - 0.005 and depth_max_env < depth +0.005: #depth_max_env > depth - 0.002:
                if depth_max_env > depth - 0.007 :# - 0.002 :#and  depth_max - depth  < 0.01:
                    print('record')
                    min_distance2 = distance2 
                    most_center_point = [center_point_y,center_point_x]
                    most_center_mask = mask

                    #most_center_depth_max =  depth_max
                    most_center_depth = [depth, depth_max]




    print(' most_center_mask')
    plt.imshow(most_center_mask)
    plt.show()


    
    
    img_copy = im.copy()

    cv2.circle(img_copy, (most_center_point[1], most_center_point[0]), 8, (255, 0, 255), 8)
    
    #print('img_copy')
    #plt.imshow(img_copy)
    #plt.show()
    
    [contours,hierarchy] = cv2.findContours(most_center_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    for j in range(len(contours)):
        if(len(contours[j]) > len(cnt)):
            cnt = contours[j]
    hull = cv2.convexHull(cnt,returnPoints = True)
    rect = cv2.minAreaRect(hull)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    #img_copy = im.copy()
    cv2.drawContours(img_copy,[box],0,(0,0,255),2)
    plt.imshow(img_copy)
    plt.show()
    print(box)
    

    return most_center_mask,most_center_point,most_center_depth,box #most_center_depth = [depth,depth_max]

In [ ]:
def find_pose(depth_im, mask, center,depth,mask_box):
    
    plt.imshow((depth_im*mask) != 0)
    plt.show()
    
    depth_intrin = [321.8862609863281, 238.18316650390625, 612.0938720703125, 611.785888671875] # cx, cy, fx, fy
    
    position = pixel_to_camera(center[::-1], depth_intrin, depth[0] )
    
    if(np.linalg.norm(mask_box[0]-mask_box[1]) > np.linalg.norm(mask_box[1]-mask_box[2])):
        rotation = math.atan2((mask_box[2]-mask_box[1])[1], (mask_box[2]-mask_box[1])[0])
        print("rotation 111")
    else:
        rotation = math.atan2((mask_box[1]-mask_box[0])[1], (mask_box[1]-mask_box[0])[0])
        print("rotation 222")
    
    depth_pixel = find_center(depth_im == depth[1]*1000)
    
    print('depth',depth)
    print('depth_pixel',depth_pixel)
    print('center',center)
    
    
    direction_max = np.array(depth_pixel) - np.array(center)
    
    angle_max = math.atan2(direction_max[1],direction_max[0])
    
    print("angle_max",math.degrees(angle_max))
    
    
   
    rotation = -rotation
    
    
    angle_difference = abs(angle_max - rotation)
    
    if angle_difference > 2*math.pi:  ### reduce more >360 to between 0 to 360
        angle_difference = angle_difference - 2* math.pi
        print("angle_difference 111")
       
    print('angle_difference,',math.degrees(angle_difference))
    
    if angle_difference > math.pi / 2.0 and angle_difference < 3 * math.pi / 2.0 : ## change gripper angle by 180 deg if it does not align with the correct height side
        rotation = rotation - math.pi
        print("rotation 211")
    elif rotation > math.pi:   ### reduce 180> x >360 to -180<x<0
        rotation = rotation - 2 * math.pi
        print("rotation 212")
   
    
    rotation = rotation + math.pi   

    
    if rotation > math.pi:  ## reduce 180> x >360 to -180<x<0
        rotation = rotation - 2 * math.pi
    
    rotation = -rotation
    
    #print('rotation,',math.degrees(rotation))
    
    pose={
            'x':position[0],
            'y':position[1],
            'z':depth[0] ,
            'yaw':rotation,
            'pitch':0,
            'normal':0
    }
    
    return pose

In [ ]:
img_index = 0
is_detect = 0
import time

In [ ]:

while (True):
    if(is_detect == 1):
        
        pose_pub = rospy.Publisher('/stone_pose', stone_pose, queue_size=10)
        
        img_index = str(img_index)
              
        
        # instance segmentation
        image = skimage.io.imread("../samples/stones/JPEGImages/"+img_index+".jpeg")
        
        depth_image = cv2.imread("../samples/stones/depth/"+img_index+".jpeg")
        depth_array = np.load("../samples/stones/depth/"+img_index+".npy")
    
        
        t = time.time()
        # Run detection
        results = model.detect([image], verbose=1)
        print('MaskRCNN time',time.time() - t)
        
        # Visualize results
        r = results[0]

        
        full_mask = np.sum(r['masks'],axis = 2)
        print('full_mask.shape',full_mask.shape)
        center = find_center(full_mask)
        
        
        img_copy = image.copy()
        cv2.circle(img_copy, (int(center[1]), int(center[0])), 8, (255, 0, 255), 8)
        
        
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    class_names, r['scores'])
        
        
        visualize.display_mask(image, r['masks'])

        #print('img_copy2')
        #plt.imshow(img_copy)
        #plt.show()
        
        
        depth_image2 = skimage.io.imread("../samples/stones/depth/"+img_index+".jpeg")
        
        mask_max, most_center_point, depth,box = find_mask(image,depth_image2 , r,center,[1,1])
        
        
        pose2 = find_pose(depth_image2 , mask_max, most_center_point, depth, box )
        
        print('finished')
        stone_pose_msg = stone_pose()
        stone_pose_msg.x = pose2['x']
        stone_pose_msg.y = pose2['y']
        stone_pose_msg.z = pose2['z']
        stone_pose_msg.yaw = pose2['yaw']
        stone_pose_msg.pitch = 0 #pose['pitch']
        stone_pose_msg.normal = [0,0,0]#pose['normal']

        print('most_center_point',most_center_point)
        pose_pub.publish(stone_pose_msg)
        print('stone_pose_msg is ', stone_pose_msg)

        
        is_detect = 0